## Importação de Bibliotecas

Nesta célula, são importadas as bibliotecas necessárias para manipulação de dados e funções do Spark.

In [0]:
from pyspark.sql.functions import monotonically_increasing_id

### A seguir será usado o comando SQL `USE CATALOG` e `USE SCHEMA` para definir o catálogo e o schema ativos.

In [0]:
spark.sql("USE CATALOG mvp")
spark.sql("USE SCHEMA bronze")

DataFrame[]

### Lendo o arquivo CSV das estações com Spark e exibindo as 10 primeiras linhas, incluindo o cabeçalho

In [0]:
df_stations = spark.read.option("header", True).csv("/Volumes/mvp/staging/dataset/stations.csv")
display(df_stations.limit(10))

region,state,city_station,id_station,lat,lon,lvl,record_first,record_last
CO,DF,BRASILIA,A001,"-15,78944444","-47,92583332","1159,54",2000-05-07,2025-05-31
NE,BA,SALVADOR,A401,"-13,01666666","-38,51666666","51,41",2000-05-13,2025-05-31
N,AM,MANAUS,A101,"-3,10333333","-60,01638888","61,25",2000-05-09,2025-05-31
SE,RJ,ECOLOGIA AGRICOLA,A601,"-22,8","-43,68333333",33,2000-05-07,2025-05-31
S,RS,PORTO ALEGRE,A801,"-30,05","-51,16666666","46,97",2000-09-22,2025-05-31
CO,GO,GOIANIA,A002,"-16,64277777","-49,21999999",770,2001-05-29,2025-05-19
CO,GO,MORRINHOS,A003,"-17,71666667","-49,1","771,42",2001-05-25,2025-05-31
CO,MS,CAMPO GRANDE,A702,"-20,45","-54,6",530,2001-09-10,2025-05-31
CO,MS,PONTA PORA,A703,"-22,5525","-55,71638888",675,2001-09-07,2025-05-31
CO,MS,TRES LAGOAS,A704,"-20,78999999","-51,71222222",313,2001-09-03,2025-05-31


### Escrita da tabela `stations` em formato Delta

Nesta célula, o DataFrame `df_stations` é persistido no metastore do Databricks como uma **tabela Delta**, utilizando sobrescrita completa.
- A tabela `stations` passa a existir como uma tabela Delta
- Os dados ficam persistidos e versionados
- A tabela pode ser consultada diretamente por SQL, notebooks ou ferramentas de BI conectadas ao Databricks

In [0]:
df_stations.write.format("delta").mode("overwrite").saveAsTable("stations")

### Comentário da tabela `mvp.bronze.stations`

Nesta célula, é definido o comentário descritivo da tabela `mvp.bronze.stations`, documentando seu propósito e principais informações armazenadas no metastore para facilitar entendimento e uso futuro.


In [0]:
spark.sql("""
    comment on table mvp.bronze.stations is
    'The table contains information about various stations, including their geographical locations and operational details. It can be used for mapping station locations, analyzing regional coverage, and tracking the operational history of each station. Key data points include the region, state, city, and latitude/longitude coordinates.'
""")

DataFrame[]

### Comentários das colunas da tabela `mvp.bronze.stations`

Nesta célula, são definidos comentários descritivos para cada coluna da tabela `mvp.bronze.stations`, documentando o significado e a finalidade de cada campo diretamente no metastore.


In [0]:
COLUMN_COMMENTS = [
    ("region", "The geographical area or zone where the station is located"),
    ("state", "State where the station is located"),
    ("city_station", "The city where the station is located"),
    ("id_station", "Unique identifier assigned to each station"),
    ("lat", "Latitude coordinate of the station location"),
    ("lon", "Longitude coordinate of the station location"),
    ("lvl", "Altitude of the station"),
    ("record_first", "Date when data recording began for the station"),
    ("record_last", "Date of the most recent station record."),
]

for column, comment in COLUMN_COMMENTS:
    spark.sql(f"comment on column mvp.bronze.stations.{column} is '{comment}'")

### Inspeção do schema e metadata da tabela `mvp.bronze.stations`

Nesta célula, é realizada a inspeção detalhada da tabela `mvp.bronze.stations` utilizando comandos `DESCRIBE`.

Primeiro, o comando `DESCRIBE EXTENDED` é executado para obter schema e metadados completos da tabela. Em seguida, é criado um identificador auxiliar (`_id`) para permitir navegar pelas linhas do resultado e extrair o bloco de informações a partir da seção **Catalog**, exibindo apenas um subconjunto relevante dessas informações.

Por fim, o comando `DESCRIBE` padrão é executado para exibir exclusivamente o schema da tabela, com foco nas colunas e seus tipos de dados.


In [0]:
df_describe = spark.sql("describe extended mvp.bronze.stations")
df_describe = df_describe.withColumn("_id", monotonically_increasing_id())
target_id = df_describe.filter("col_name = 'Catalog'").select("_id").first()._id

table_describe = df_describe.filter(f"_id >= {target_id}").limit(9)
display(table_describe.drop("_id"))

display(spark.sql("describe mvp.bronze.stations"))

col_name,data_type,comment
Catalog,mvp,
Database,bronze,
Table,stations,
Created Time,Sun Dec 21 20:26:50 UTC 2025,
Last Access,UNKNOWN,
Created By,Spark,
Statistics,"19107 bytes, 615 rows",
Type,MANAGED,
Comment,"The table contains information about various stations, including their geographical locations and operational details. It can be used for mapping station locations, analyzing regional coverage, and tracking the operational history of each station. Key data points include the region, state, city, and latitude/longitude coordinates.",


col_name,data_type,comment
region,string,The geographical area or zone where the station is located
state,string,State where the station is located
city_station,string,The city where the station is located
id_station,string,Unique identifier assigned to each station
lat,string,Latitude coordinate of the station location
lon,string,Longitude coordinate of the station location
lvl,string,Altitude of the station
record_first,string,Date when data recording began for the station
record_last,string,Date of the most recent station record.


### Leitura dos dados meteorológicos

Nesta célula, os arquivos CSV de dados meteorológicos são carregados para um DataFrame Spark utilizando um padrão de nome (`weather_*_filtered.csv`), permitindo a leitura conjunta de múltiplos arquivos de forma automática. Em seguida, é exibida uma amostra dos dados para verificação do conteúdo carregado.


In [0]:
    #Abrindo CSVs
df_weather_data = (
    spark.read
    .option("header", True)
    .csv(f"/Volumes/mvp/staging/dataset/weather_*_filtered.csv")
)

display(df_weather_data.limit(10))

DATA (YYYY-MM-DD),Hora UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)",ESTACAO
2024-01-01,0000 UTC,0.0,885.7,885.7,885.3,null,22.0,18.2,22.6,21.8,18.6,18.2,81.0,77.0,79.0,311.0,2.8,1.5,A001
2024-01-01,0100 UTC,0.0,886.7,886.7,885.7,null,21.2,18.6,22.0,21.1,18.7,18.1,85.0,79.0,85.0,315.0,2.9,1.5,A001
2024-01-01,0200 UTC,0.0,887.1,887.2,886.7,null,20.9,18.8,21.5,20.9,18.9,18.7,88.0,85.0,88.0,305.0,3.3,1.1,A001
2024-01-01,0300 UTC,0.0,887.1,887.4,887.1,null,20.9,18.3,21.1,20.8,18.9,18.3,88.0,85.0,85.0,304.0,3.3,1.5,A001
2024-01-01,0400 UTC,0.0,886.4,887.1,886.4,null,20.5,18.6,21.0,20.5,18.6,18.3,89.0,85.0,89.0,315.0,4.0,2.0,A001
2024-01-01,0500 UTC,0.0,886.1,886.4,886.1,null,20.4,18.9,20.5,20.3,18.9,18.6,91.0,89.0,91.0,310.0,4.0,1.5,A001
2024-01-01,0600 UTC,0.0,885.5,886.1,885.4,null,19.6,18.7,20.4,19.6,18.9,18.6,94.0,91.0,94.0,309.0,3.0,1.1,A001
2024-01-01,0700 UTC,0.0,885.2,885.5,885.2,null,20.0,19.0,20.1,19.4,19.2,18.5,95.0,94.0,94.0,333.0,2.6,1.4,A001
2024-01-01,0800 UTC,0.0,885.5,885.5,885.1,null,20.1,19.1,20.1,19.7,19.2,18.8,95.0,94.0,94.0,308.0,2.6,1.2,A001
2024-01-01,0900 UTC,0.0,885.7,885.8,885.5,4.3,19.6,18.8,20.1,19.6,19.1,18.8,96.0,94.0,95.0,312.0,2.4,1.1,A001


### Padronização dos nomes das colunas

Nesta célula, os nomes das colunas do DataFrame `df_weather_data` são padronizados por meio da substituição de caracteres especiais e espaços por sublinhados (`_`). Esse processo garante maior compatibilidade com Spark SQL, facilita consultas e evita problemas com caracteres não permitidos em nomes de colunas. Em seguida, é exibida uma amostra dos dados com os nomes das colunas já normalizados.


In [0]:
# Substituindo caracteres especiais não desejados
df_replaced_weather_data = df_weather_data.toDF(*[col.replace(' ', '_').replace('(', '_').replace(')', '_').replace('-', '_').replace('.', '_').replace('°', '_').replace('(', '_').replace(')', '_').replace(',', '_') for col in df_weather_data.columns])

display(df_replaced_weather_data.limit(10))

DATA__YYYY_MM_DD_,Hora_UTC,PRECIPITAÇÃO_TOTAL__HORÁRIO__mm_,PRESSAO_ATMOSFERICA_AO_NIVEL_DA_ESTACAO__HORARIA__mB_,PRESSÃO_ATMOSFERICA_MAX_NA_HORA_ANT___AUT___mB_,PRESSÃO_ATMOSFERICA_MIN__NA_HORA_ANT___AUT___mB_,RADIACAO_GLOBAL__KJ/m²_,TEMPERATURA_DO_AR___BULBO_SECO__HORARIA___C_,TEMPERATURA_DO_PONTO_DE_ORVALHO___C_,TEMPERATURA_MÁXIMA_NA_HORA_ANT___AUT____C_,TEMPERATURA_MÍNIMA_NA_HORA_ANT___AUT____C_,TEMPERATURA_ORVALHO_MAX__NA_HORA_ANT___AUT____C_,TEMPERATURA_ORVALHO_MIN__NA_HORA_ANT___AUT____C_,UMIDADE_REL__MAX__NA_HORA_ANT___AUT___%_,UMIDADE_REL__MIN__NA_HORA_ANT___AUT___%_,UMIDADE_RELATIVA_DO_AR__HORARIA__%_,VENTO__DIREÇÃO_HORARIA__gr______gr__,VENTO__RAJADA_MAXIMA__m/s_,VENTO__VELOCIDADE_HORARIA__m/s_,ESTACAO
2024-01-01,0000 UTC,0.0,885.7,885.7,885.3,null,22.0,18.2,22.6,21.8,18.6,18.2,81.0,77.0,79.0,311.0,2.8,1.5,A001
2024-01-01,0100 UTC,0.0,886.7,886.7,885.7,null,21.2,18.6,22.0,21.1,18.7,18.1,85.0,79.0,85.0,315.0,2.9,1.5,A001
2024-01-01,0200 UTC,0.0,887.1,887.2,886.7,null,20.9,18.8,21.5,20.9,18.9,18.7,88.0,85.0,88.0,305.0,3.3,1.1,A001
2024-01-01,0300 UTC,0.0,887.1,887.4,887.1,null,20.9,18.3,21.1,20.8,18.9,18.3,88.0,85.0,85.0,304.0,3.3,1.5,A001
2024-01-01,0400 UTC,0.0,886.4,887.1,886.4,null,20.5,18.6,21.0,20.5,18.6,18.3,89.0,85.0,89.0,315.0,4.0,2.0,A001
2024-01-01,0500 UTC,0.0,886.1,886.4,886.1,null,20.4,18.9,20.5,20.3,18.9,18.6,91.0,89.0,91.0,310.0,4.0,1.5,A001
2024-01-01,0600 UTC,0.0,885.5,886.1,885.4,null,19.6,18.7,20.4,19.6,18.9,18.6,94.0,91.0,94.0,309.0,3.0,1.1,A001
2024-01-01,0700 UTC,0.0,885.2,885.5,885.2,null,20.0,19.0,20.1,19.4,19.2,18.5,95.0,94.0,94.0,333.0,2.6,1.4,A001
2024-01-01,0800 UTC,0.0,885.5,885.5,885.1,null,20.1,19.1,20.1,19.7,19.2,18.8,95.0,94.0,94.0,308.0,2.6,1.2,A001
2024-01-01,0900 UTC,0.0,885.7,885.8,885.5,4.3,19.6,18.8,20.1,19.6,19.1,18.8,96.0,94.0,95.0,312.0,2.4,1.1,A001


### Escrita da tabela `weather_data` em formato Delta

In [0]:
df_replaced_weather_data.write.format("delta").mode("overwrite").saveAsTable("weather_data")

### Comentário da tabela `mvp.bronze.weather_data`

In [0]:
spark.sql("""
    comment on table mvp.bronze.weather_data is
    'The table contains hourly weather data collected from various stations. It includes information such as temperature, humidity, precipitation, and wind conditions. Possible use cases include analyzing weather patterns, conducting climate research, and supporting agricultural planning by understanding local weather conditions.'
""")

DataFrame[]

### Comentários das colunas da tabela `mvp.bronze.weather_data`


In [0]:
COLUMN_COMMENTS = [
    ("DATA__YYYY_MM_DD_", "Date of the observation recorded in YYYY-MM-DD format."),
    ("Hora_UTC", "Hour of the observation recorded in Coordinated Universal Time (UTC)"),
    ("PRECIPITAÇÃO_TOTAL__HORÁRIO__mm_", "Total hourly precipitation measured in millimeters"),
    ("PRESSAO_ATMOSFERICA_AO_NIVEL_DA_ESTACAO__HORARIA__mB_", "Hourly atmospheric pressure at the station level, measured in millibars (mB)"),
    ("PRESSÃO_ATMOSFERICA_MAX_NA_HORA_ANT___AUT___mB_", "Maximum atmospheric pressure measured in the previous hour, recorded automatically, given in millibars (mB)"),
    ("PRESSÃO_ATMOSFERICA_MIN__NA_HORA_ANT___AUT___mB_", "Minimum atmospheric pressure recorded during the previous hour, measured in millibars."),
    ("RADIACAO_GLOBAL__KJ/m²_", "Amount of global radiation recorded in kilojoules per square meter"),
    ("TEMPERATURA_DO_AR___BULBO_SECO__HORARIA___C_", "Hourly dry-bulb air temperature recorded in degrees Celsius."),
    ("TEMPERATURA_DO_PONTO_DE_ORVALHO___C_", "Dew point temperature in Celsius, representing the air temperature at which condensation occurs."),
    ("TEMPERATURA_MÁXIMA_NA_HORA_ANT___AUT____C_", "Highest air temperature recorded in the previous hour, measured in degrees Celsius"),
    ("TEMPERATURA_MÍNIMA_NA_HORA_ANT___AUT____C_", "Lowest air temperature recorded in the preceding hour, in Celsius"),
    ("TEMPERATURA_ORVALHO_MAX__NA_HORA_ANT___AUT____C_", "Highest dew point temperature recorded during the previous hour (°C)"),
    ("TEMPERATURA_ORVALHO_MIN__NA_HORA_ANT___AUT____C_", "Minimum dew point temperature recorded in the previous hour, in Celsius"),
    ("UMIDADE_REL__MAX__NA_HORA_ANT___AUT___%_", "Maximum relative humidity recorded in the previous hour as measured automatically, expressed in percent."),
    ("UMIDADE_REL__MIN__NA_HORA_ANT___AUT___%_", "Lowest relative humidity recorded automatically in the previous hour, expressed as percentage"),
    ("UMIDADE_RELATIVA_DO_AR__HORARIA__%_", "Percentage of relative humidity measured for each hour"),
    ("VENTO__DIREÇÃO_HORARIA__gr______gr__", "Hourly wind direction measured in degrees"),
    ("VENTO__RAJADA_MAXIMA__m/s_", "Highest recorded wind gust speed during the hour, measured in meters per second"),
    ("VENTO__VELOCIDADE_HORARIA__m/s_", "Measured wind speed per hour, expressed in meters per second"),
    ("ESTACAO", "Station identifier where the hourly weather data was recorded"),
]

for column, comment in COLUMN_COMMENTS:
    spark.sql(f"comment on column mvp.bronze.weather_data.`{column}` is '{comment}'")

### Inspeção do schema e metadata da tabela `mvp.bronze.weather_data`

In [0]:
df_describe = spark.sql("describe extended mvp.bronze.weather_data")
df_describe = df_describe.withColumn("_id", monotonically_increasing_id())
target_id = df_describe.filter("col_name = 'Catalog'").select("_id").first()._id

table_describe = df_describe.filter(f"_id >= {target_id}").limit(9)
display(table_describe.drop("_id"))

display(spark.sql("describe mvp.bronze.weather_data"))

col_name,data_type,comment
Catalog,mvp,
Database,bronze,
Table,weather_data,
Created Time,Sun Dec 21 20:27:10 UTC 2025,
Last Access,UNKNOWN,
Created By,Spark,
Statistics,"17449608 bytes, 1095216 rows",
Type,MANAGED,
Comment,"The table contains hourly weather data collected from various stations. It includes information such as temperature, humidity, precipitation, and wind conditions. Possible use cases include analyzing weather patterns, conducting climate research, and supporting agricultural planning by understanding local weather conditions.",


col_name,data_type,comment
DATA__YYYY_MM_DD_,string,Date of the observation recorded in YYYY-MM-DD format.
Hora_UTC,string,Hour of the observation recorded in Coordinated Universal Time (UTC)
PRECIPITAÇÃO_TOTAL__HORÁRIO__mm_,string,Total hourly precipitation measured in millimeters
PRESSAO_ATMOSFERICA_AO_NIVEL_DA_ESTACAO__HORARIA__mB_,string,"Hourly atmospheric pressure at the station level, measured in millibars (mB)"
PRESSÃO_ATMOSFERICA_MAX_NA_HORA_ANT___AUT___mB_,string,"Maximum atmospheric pressure measured in the previous hour, recorded automatically, given in millibars (mB)"
PRESSÃO_ATMOSFERICA_MIN__NA_HORA_ANT___AUT___mB_,string,"Minimum atmospheric pressure recorded during the previous hour, measured in millibars."
RADIACAO_GLOBAL__KJ/m²_,string,Amount of global radiation recorded in kilojoules per square meter
TEMPERATURA_DO_AR___BULBO_SECO__HORARIA___C_,string,Hourly dry-bulb air temperature recorded in degrees Celsius.
TEMPERATURA_DO_PONTO_DE_ORVALHO___C_,string,"Dew point temperature in Celsius, representing the air temperature at which condensation occurs."
TEMPERATURA_MÁXIMA_NA_HORA_ANT___AUT____C_,string,"Highest air temperature recorded in the previous hour, measured in degrees Celsius"
